# L402: A Beginner's Guide to Internet-Native Paywalls

L402 is a decentralized payment protocol built on the Lightning Network. It allows servers to require payment for access to resources, creating internet-native paywalls. This post will walk you through the key concepts and show you how to implement L402 in your own applications.

The following sequence diagram shows the flow of L402 authentication.

```mermaid
sequenceDiagram
    Client->>Server: Request access to HTTP resource
    Server-->Server: Check client credentials (invalid)
    Server-->Server: Generate challenge (macaroon + invoice)
    Server-->>Client: Return HTTP 402 status code & challenge 
    Client->>Client: Complete challenge & obtain preimage
    Client->>Server: Request access L402 Authentication header (macaroon + preimage)
    Server->>Server: Verify credentials and proof of completion for the linked challenge
    Server-->>Client: Serve the requested resource
```

The sequence diagram above outlines the L402 protocol's interaction flow. It starts with the client requesting access to a protected resource. The server then returns an `HTTP 402 Payment required` status code and an invoice to the client. When the client pays the invoice, it will receive a preimage as proof of payment. Requesting access again with the preimage, the server will grant access to the resource.

Now, let's explore the key concepts involved in this protocol:

* Macaroons
* Invoices
* Preimages



## Key Concepts

### Invoices

L402 uses Lightning invoices to handle payments. The server includes an invoice in its 402 response. The client pays this invoice to gain access.

Example of encoded invoice:

```
lnbc20m1pvjluezpp5qqqsyqcyq5rqwzqfqqqsyqcyq5rqwzqfqqqsyqcyq5rqwzqfqypqhp58yjmdan79s6qqdhdzgynm4zwqd5d7xmw5fk98klysy043l2ahrqsfpp3qjmp7lwpagxun9pygexvgpjdc4jdj85fr9yq20q82gphp2nflc7jtzrcazrra7wwgzxqc8u7754cdlpfrmccae92qgzqvzq2ps8pqqqqqqpqqqqq9qqqvpeuqafqxu92d8lr6fvg0r5gv0heeeqgcrqlnm6jhphu9y00rrhy4grqszsvpcgpy9qqqqqqgqqqqq7qqzqj9n4evl6mr5aj9f58zp6fyjzup6ywn3x6sk8akg5v4tgn2q8g4fhx05wf6juaxu9760yp46454gpg5mtzgerlzezqcqvjnhjh8z3g2qqdhhwkj
```

Decoded invoice:

```
{
    "destination": "03e7156ae33b0a208d0744199163177e909e80176e55d97a2f221ede0f934dd9ad",
    "payment_hash": "0001020304050607080900010203040506070809000102030405060708090102",
    "num_satoshis": "2000000",
    "timestamp": "1496314658",
    "expiry": "3600",
    "description": "",
    ...
}
```


### Macaroons
Macaroons are bearer tokens that enable decentralized authorization. They contain caveats that specify constraints on their usage.

Macaroon example:

```
{
  "ID": "000035cf3da4dfdefa01a385965...",
  "version": 0,
  "payment_hash": "35cf3da4dfdefa01a385965...",
  "token_id": "7133548b39c094b83120052b106...",
  "location": "fewsats.com",
  "caveats": [
    "expires_at=2024-06-14T01:38:46Z"
  ]
}
```

### Preimages
Each Lightning invoice has a corresponding preimage, which is revealed when the invoice is paid. In L402, the preimage serves as proof of payment. 

Preimage example:

```
e451ccb4f1fb9d83d5c4f03a902e4cc651ef0b71999e764b152881a5f3260d50
```

## Building L402 Components Step-by-Step

Now let's build the components of the L402 protocol step-by-step. We'll start with the basics and gradually build up to a complete example.

### 1. Invoice Generation

1. **Invoice Creation:** An invoice is generated specifying the amount, currency, and a description that ties it to the specific content access request. This ensures that the payment is directly linked to the content being accessed.

2. **Encoding Payment Request:** The invoice is encoded into a `payment_request` string. This string is a compact, URL-safe format that can be easily transmitted and handled by payment systems.

3. **Linking to Macaroon:** The `payment_hash` generated during the invoice creation is used in the macaroon's identifier. This establishes a secure link between the payment and the authorization token, ensuring that each payment corresponds to a specific access permission.

4. **Transmission Preparation:** The `payment_request` is prepared for transmission alongside the macaroon, enabling the client to make a payment directly related to the resource they wish to access.



In [12]:
# We need to use an Invoice Provider, we'll mock one for this example
class MockInvoiceProvider():
    def create_invoice(self, amount, currency, description):
        mock_invoice = "lnbcrt1u1p3d23dkpp58r92m0s0vyfdnd3caxhzgvu006dajv9r8pcspknhvezw26t9e8qsdq5g9kxy7fqd9h8vmmfvdjscqzpgxqyz5vqsp59efe44rg6cjl3xwh9glgx4ztcgwtg5l8uhry2v9v7s0zn2wpaz2s9qyyssq2z799an4pt4wtfy8yrk5ee0qqj7w5a74prz5tm8rulwez08ttlaz9xx7eqw7fe94y7t0600d03k55fyguyj24nd9tjmx6sf7dsxkk4gpkyenl8"
        mock_payment_hash = "38caadbe0f6112d9b638e9ae24338f7e9bd930a3387100da776644e56965c9c1"
        return mock_invoice, mock_payment_hash

# The provider accepts an amount, currency, and description
amount = 100
currency = "USD"
description = "L402 Challenge: Downloading a file from fewsats.com"

invoice, payment_hash  = MockInvoiceProvider().create_invoice(
    amount, currency, f"L402 Challenge: {description}",
)
invoice, payment_hash

('lnbcrt1u1p3d23dkpp58r92m0s0vyfdnd3caxhzgvu006dajv9r8pcspknhvezw26t9e8qsdq5g9kxy7fqd9h8vmmfvdjscqzpgxqyz5vqsp59efe44rg6cjl3xwh9glgx4ztcgwtg5l8uhry2v9v7s0zn2wpaz2s9qyyssq2z799an4pt4wtfy8yrk5ee0qqj7w5a74prz5tm8rulwez08ttlaz9xx7eqw7fe94y7t0600d03k55fyguyj24nd9tjmx6sf7dsxkk4gpkyenl8',
 '38caadbe0f6112d9b638e9ae24338f7e9bd930a3387100da776644e56965c9c1')


### 2. Macaroon Generation

1. **Key Generation:** Unique keys (`token_id` and `root_key`) are generated to uniquely identify and secure each macaroon.

2. **Setting the Location:** The `location` specifies the URL of the content being accessed. It links the macaroon to the content.

3. **Linking Payment Hash:** The `payment_hash` from the corresponding invoice is included in the macaroon's identifier. This links the macaroon to a specific payment, ensuring that the access granted is tied to a completed transaction.

4. **Serialization:** The macaroon is serialized into a string format, making it suitable for transmission over the network.


In [13]:
from pymacaroons import Macaroon, MACAROON_V2
from binascii import unhexlify
import os, struct

token_id = os.urandom(32)
root_key = os.urandom(32)
version = 0
# generate an ID from the token & root key, details aren't important now
identifier = struct.pack(">H32s32s", version, unhexlify(payment_hash), token_id)
url = "https://api.fewsats.com/v0/storage/download/b0dfa667-bafe-4648-a371-b94c8a87da65"
mac = Macaroon(
    version=MACAROON_V2,
    location=url,
    identifier=identifier,
    key=root_key
)
macaroon = mac.serialize()
macaroon

'AgFQaHR0cHM6Ly9hcGkuZmV3c2F0cy5jb20vdjAvc3RvcmFnZS9kb3dubG9hZC9iMGRmYTY2Ny1iYWZlLTQ2NDgtYTM3MS1iOTRjOGE4N2RhNjUCQgAAOMqtvg9hEtm2OOmuJDOPfpvZMKM4cQDad2ZE5WllycHsw9WR14p-G3_rsRP-Pny6KBJRyIvNEYu_F01u7lIEhwAABiC_Tv6Et72-N8ptQYexKAQAZFheJxqS_jz6A8YlpZjwOA'

### 3. Creating and Sending the Challenge

Let's combine the macaroon and invoice into a challenge and send it to the client. This challenge, encapsulating authentication and payment details, is transmitted via an HTTP 402 (Payment Required) response.

In [14]:
from flask import Response

challenge = f'L402 macaroon="{macaroon}", invoice="{invoice}"'
response = Response(status=402)

response.headers['WWW-Authenticate'] = challenge
response

<Response 0 bytes [402 PAYMENT REQUIRED]>

### 4. Client Parses the Challenge

After receiving the challenge, the client needs to parse it to extract the macaroon and invoice. 

In [15]:
import re, requests

def parse_l402_challenge(challenge):
    macaroon = re.search(r'macaroon="([^ ]+)"', challenge).group(1)
    invoice = re.search(r'invoice="([^ ]+)"', challenge).group(1)
    return macaroon, invoice

response = requests.get("https://api.fewsats.com/v0/storage/download/b0dfa667-bafe-4648-a371-b94c8a87da65")
# Extract challenge from server response
challenge = response.headers.get('WWW-Authenticate')
# We extract the macaroon & preimage simply using regex
macaroon, invoice = parse_l402_challenge(challenge)
macaroon, invoice


('AgELZmV3c2F0cy5jb20CQgAAhdYJswU80RmRx4Pf16TGIjEFRIOp6ZrQRqb5y7OTppm+k7XiLYczm+7aPlzRgZ+AxonD+wAkUhuXD4j6rl2l2wACLGZpbGVfaWQ9YjBkZmE2NjctYmFmZS00NjQ4LWEzNzEtYjk0YzhhODdkYTY1AAIfZXhwaXJlc19hdD0yMDI0LTA3LTExVDEwOjQwOjI3WgAABiCdDIKl9Lu6JpjsdQ83+yzto0V5sWuUW7Z5hKwwWj+/xw==',
 'lnbc150n1pnxs2vupp5shtqnvc98ng3nyw8s00a0fxxygcs23yr485e45zx5muuhvun56vsdpygejhwumpw3ejqnp5xqezqsmgv9kxcetwvajscqzzsxqyz5vqsp5rdnl07nnxmjzay5l0gqj9w52tcg6psheqfxjx4940p26klgtmqnq9qyyssquvewm0zw9gsl3fptgjhz2ytlgyrxfwzm6zln6kxfl3ks6nx9mm0rd97hypggegf20arx30wvucgptr6jzsegvgpr7unx4fnl9nrpsnspnp3rf3')

### 5. Paying the Invoice

After parsing the challenge, the client uses a mock preimage provider to simulate the payment of the invoice and obtain a preimage. This preimage serves as proof of payment, which is necessary for the client to authenticate subsequent requests to access the protected resource.

In [16]:
# Simulate payment and obtain preimage using a mock preimage provider
class MockPreimageProvider():
    def get_preimage(self, invoice):
        return "2f84e22556af9919f695d7761f404e98ff98058b7d32074de8c0c83bf63eecd7"
    
preimage_provider = MockPreimageProvider()
preimage = preimage_provider.get_preimage(invoice)
preimage

'2f84e22556af9919f695d7761f404e98ff98058b7d32074de8c0c83bf63eecd7'

### 6. Sending Authenticated Request with Macaroon and Preimage

After obtaining the preimage, the client sends an authenticated request by combining the macaroon and preimage in the request headers.

In [17]:
import requests

# Send authenticated request with macaroon and preimage
headers = {
    'Authorization': f'L402 {macaroon}:{preimage}'
}

response = requests.get(url, headers=headers)
response

<Response [402]>

The response is still 402 because we made up the `preimage` so the credentials are not valid. But you get the point. Let's now put it all together.

## Putting it All Together
Now let's see the complete L402 flow in action by running the Flask server and creating a client that interacts with it.
First, let's start the Flask server:

### Flask Server

We'll use the `Flask_l402_decorator` to protect our endpoint, which takes an `Authenticator` instance and a pricing function as arguments.

The Authenticator requires:

* `InvoiceProvider` to create invoices and a
* `MacaroonService` to store and retrieve macaroon root keys to validate them


In [18]:
# Let's install the L402 library that we'll use for both client & server
# Google colab has an issue with blinker so we uninstall it first.
# See more: https://github.com/googlecolab/colabtools/issues/3976

# If running in Colab RUN the following command.
# apt-get -qq remove python3-blinker

# If you have issues outside of Colab, run the following command.
# See more: https://github.com/embedchain/embedchain/issues/506
# %pip install --ignore-installed -Uqq l402

%pip install -Uqq l402

Note: you may need to restart the kernel to use updated packages.


In [19]:

from flask import Flask, jsonify
from l402.server import Authenticator, Flask_l402_decorator
from l402.server.macaroons import MacaroonService

# We defined again the mocks here because previous examples were not async
# and Flask requires async functions to work with them
class MockInvoiceProvider():
    async def create_invoice(self, amount, currency, description):
        mock_invoice = "lnbcrt1u1p3d23dkpp58r92m0s0vyfdnd3caxhzgvu006dajv9r8pcspknhvezw26t9e8qsdq5g9kxy7fqd9h8vmmfvdjscqzpgxqyz5vqsp59efe44rg6cjl3xwh9glgx4ztcgwtg5l8uhry2v9v7s0zn2wpaz2s9qyyssq2z799an4pt4wtfy8yrk5ee0qqj7w5a74prz5tm8rulwez08ttlaz9xx7eqw7fe94y7t0600d03k55fyguyj24nd9tjmx6sf7dsxkk4gpkyenl8"
        mock_payment_hash = "38caadbe0f6112d9b638e9ae24338f7e9bd930a3387100da776644e56965c9c1"
        return mock_invoice, mock_payment_hash

class MockMacaroonService(MacaroonService):
    def __init__(self):
        self._store = {}

    async def insert_root_key(self, token_id, root_key, macaroon):
        self._store[token_id] = (root_key, macaroon)

    async def get_root_key(self, token_id):
        return self._store[token_id][0] if token_id in self._store else None


authenticator = Authenticator(
    location="https://api.fewsats.com", # <- this will be added to the created macaroons
    invoice_provider=MockInvoiceProvider(),
    macaroon_service=MockMacaroonService()
)

def pricing_func(request):
    return 1, "USD", "L402 protected endpoint"

app = Flask(__name__)

@app.route('/protected')
@Flask_l402_decorator(authenticator, pricing_func)
def protected_endpoint():
    return jsonify({"message": "Access granted to protected endpoint"})


### Client

The L402 package provides an `httpx` drop-in replacement. You only need to configure it with a `preimage_provider` and a `credentials_service`. All you existing codebase using `httpx` will work out of the box.

The `credentials_service`  acts as a store for the preimages obtained after a payment. It is not an essential part of the L402 protocol but without it, we would need to pay again for each request.

In [20]:
from l402.client import httpx

# We store the credentials in a simple dict
class MockCredentialsService:
    def __init__(self):
        self._credentials = {}
    async def store(self, credentials):
        self._credentials[credentials.location] = credentials

    async def get(self, location):
        return self._credentials.get(location, None)

# We will return a hardcoded preimage for the input invoice
class MockPreimageProvider():
    async def get_preimage(self, invoice):
        # Preimage for "lnbcrt1u1p3d23dkpp58r92m0s0vyfdnd3caxhzgvu006dajv9r8pcspknhvezw26t9e8qsdq5g9kxy7fqd9h8vmmfvdjscqzpgxqyz5vqsp59efe44rg6cjl3xwh9glgx4ztcgwtg5l8uhry2v9v7s0zn2wpaz2s9qyyssq2z799an4pt4wtfy8yrk5ee0qqj7w5a74prz5tm8rulwez08ttlaz9xx7eqw7fe94y7t0600d03k55fyguyj24nd9tjmx6sf7dsxkk4gpkyenl8"
        return "2f84e22556af9919f695d7761f404e98ff98058b7d32074de8c0c83bf63eecd7"

httpx.configure(
    preimage_provider=MockPreimageProvider(),
    credentials_service=MockCredentialsService()
)

In [21]:
# Server: Run Flask in a thread
import threading
from time import sleep
threading.Thread(target=app.run).start()

sleep(2) # wait for the server to start

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [22]:

# Client: Send a request to the server
import httpx as base_httpx
async with base_httpx.AsyncClient() as client:
    response = await client.get("http://127.0.0.1:5000/protected")
    print(response)
    print(response.headers.get('WWW-Authenticate'))
    print(response.text)

127.0.0.1 - - [11/Jun/2024 12:40:32] "GET /protected HTTP/1.1" 402 -
127.0.0.1 - - [11/Jun/2024 12:40:32] "GET /protected HTTP/1.1" 402 -
127.0.0.1 - - [11/Jun/2024 12:40:32] "GET /protected HTTP/1.1" 200 -


<Response [402 PAYMENT REQUIRED]>
L402 macaroon="AgEXaHR0cHM6Ly9hcGkuZmV3c2F0cy5jb20CQgAAOMqtvg9hEtm2OOmuJDOPfpvZMKM4cQDad2ZE5WllycFb74udcOZziMdzDlgyNZKfZH7RS_V1Ubk3pA2h6OduqwAABiDWpNz2YGqBt60g1vDAYGpzPfM3QoxCsoZEg2Y7sMe9aQ", invoice="lnbcrt1u1p3d23dkpp58r92m0s0vyfdnd3caxhzgvu006dajv9r8pcspknhvezw26t9e8qsdq5g9kxy7fqd9h8vmmfvdjscqzpgxqyz5vqsp59efe44rg6cjl3xwh9glgx4ztcgwtg5l8uhry2v9v7s0zn2wpaz2s9qyyssq2z799an4pt4wtfy8yrk5ee0qqj7w5a74prz5tm8rulwez08ttlaz9xx7eqw7fe94y7t0600d03k55fyguyj24nd9tjmx6sf7dsxkk4gpkyenl8"
Payment Required


In [23]:
async with httpx.AsyncClient() as client:
    response = await client.get("http://127.0.0.1:5000/protected")
    print(response.text)

{"message":"Access granted to protected endpoint"}

